# Mapbox Take Home (Matching Businesses from two input files)

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon, shape
import json

We will load the data sets (ds1 and ds2). Clean up data will follow.

In [2]:
ds1 = pd.read_csv('../data/data_source_1.csv')
ds2 = pd.read_csv('../data/data_source_2.csv')


def parse_geometry(geom_str):
    if pd.isna(geom_str):
        return None
    try:
        geom_dict = json.loads(geom_str)
        return shape(geom_dict)
    except (json.JSONDecodeError, TypeError) as e:
        print(f"Failed to parse geometry: {geom_str}")
        print(f"Error: {str(e)}")
        return None

def process_geometry_columns(df):
    """Process geometry columns in a dataframe"""
    if 'geometry' in df.columns:
        df['geometry'] = df['geometry'].apply(parse_geometry)
    return df

# Process geometry columns
ds1 = process_geometry_columns(ds1)
ds2 = process_geometry_columns(ds2)

# Create a function to extract first 5 digits
def extract_zip(postcode):
    if pd.isna(postcode):
        return None
    # Convert to string and take first 5 characters
    return str(postcode)[:5]

# Create the new zip column
ds1['zip'] = ds1['postcode'].apply(extract_zip)
ds2['zip'] = ds2['postcode'].apply(extract_zip)


ds1.describe(include="all")

,id,provider,name,address,geometry,categories,city,country,postcode,mapbox_id,zip
count,100.000000,100,100,100,100,100,0.0,100,1.000000e+02,100,100
unique,NaN,1,27,100,99,18,NaN,1,NaN,100,38
top,NaN,Provider 1,Starbucks,137 W Ellendale Ave,POINT (-96.68496482 32.84827516),cafe;coffee;coffee_shop;food;food_and_drink,NaN,US,NaN,dXJuOm1ieHBvaTpkOTZiZDM4MC1hMjg1LTQ2ZTEtYWVhYS...,75206
freq,NaN,100,40,1,2,40,NaN,100,NaN,1,6
mean,50.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.459893e+08,NaN,NaN
std,29.011492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.557409e+08,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.013200e+04,NaN,NaN
25%,25.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.521350e+04,NaN,NaN
50%,50.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.523500e+04,NaN,NaN
75%,75.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.520658e+08,NaN,NaN


In [3]:
ds2.describe(include="all")

,id,provider,name,address,geometry,categories,city,country,postcode,mapbox_id,zip
count,100,100,100,100,100,95,90,100,100.000000,100,100
unique,100,1,18,99,97,17,1,1,NaN,100,38
top,a1,Provider 2,Starbucks,8008 Herb Kelleher Way,POINT (-123.31257629394531 44.930686950683594),bakery;cafe;coffee;coffee_shop;food;food_and_d...,Dallas,US,NaN,dXJuOm1ieHBvaTpkOTZiZDM4MC1hMjg1LTQ2ZTEtYWVhYS...,75231
freq,1,100,41,2,2,39,90,100,NaN,1,6
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73642.880000,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9192.365244,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30132.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75207.750000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75220.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75235.250000,NaN,NaN


In [ ]:
from sentence_transformers import SentenceTransformer
import torch
from typing import List, Optional, Union, Tuple, Dict
import numpy as np
import pandas as pd

def get_embedding_model(model_name: str = 'all-MiniLM-L6-v2', device: Optional[str] = None) -> SentenceTransformer:
    """
    Initialize and return a sentence transformer model.
    
    Args:
        model_name (str): Name of the pre-trained model to use
        device (str, optional): Device to run the model on ('cuda', 'cpu', or None for auto-detect)
    
    Returns:
        SentenceTransformer: Initialized sentence transformer model
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Initialize the model using the exact pattern from the notebook
    model =  SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # Move to specified device
    model.to(device)
    return model

def get_embeddings(model: SentenceTransformer, texts: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Get embeddings for a list of texts using the sentence transformer model.
    
    Args:
        model (SentenceTransformer): Initialized sentence transformer model
        texts (list): List of text strings to embed
        batch_size (int): Batch size for processing
    
    Returns:
        np.ndarray: Array of embeddings
    """
    return model.encode(texts, batch_size=batch_size, show_progress_bar=True)


def add_embedding_columns(df: pd.DataFrame, model: SentenceTransformer, columns: List[str] = ['name', 'address', 'categories']) -> pd.DataFrame:
    """
    Add embedding columns for specified text columns in the dataframe.
    Each embedding is stored as a single column containing the full vector.
    
    Args:
        df (pd.DataFrame): Input dataframe
        model (SentenceTransformer): Initialized sentence transformer model
        dataset_name (str): Name of the dataset ('ds1' or 'ds2')
        columns (List[str]): List of column names to create embeddings for
    
    Returns:
        pd.DataFrame: DataFrame with new embedding columns
    """
    for col in columns:
        if col not in df.columns:
            print(f"Warning: Column {col} not found in dataframe")
            continue
            
        # Handle missing values
        df[col] = df[col].fillna('')
        
        # Create embeddings
        embeddings = get_embeddings(model, df[col].tolist())
        
        # Add embedding as a single column containing the full vector with dataset suffix
        df[f'{col}_embedding'] = [emb for emb in embeddings]
    
    return df

In [5]:
# Initialize the model
model = get_embedding_model()

# Or specify which columns to embed
df1_with_embeddings = add_embedding_columns(ds1, model)
df2_with_embeddings = add_embedding_columns(ds2, model)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

### Join data analysis on mapbox_id

Understanding how to generate pairs and also look more closely at the data

In [6]:
# Join the dataframes on mapbox_id
merged_df = pd.merge(df1_with_embeddings, df2_with_embeddings, on='mapbox_id', how='inner', suffixes=('_ds1', '_ds2'))

# Display the matching cases
print("Number of matching cases:", len(merged_df))
print("\nMatching cases:")
print(merged_df[['name_ds1', 'name_ds2', 'address_ds1', 'address_ds2', 'zip_ds1', 'zip_ds2', 'mapbox_id']].head())

# Show some statistics about the matches
print("\nStatistics about matches:")
print("Unique business names in matches:", merged_df['name_ds1'].nunique())
print("Unique business zips in matches:", merged_df['zip_ds1'].nunique())
print("\nMost common business names in matches:")
print(merged_df['name_ds1'].value_counts().head(10))

Number of matching cases: 100

Matching cases:
          name_ds1                      name_ds2              address_ds1  \
0        Starbucks                     Starbucks      137 W Ellendale Ave   
1        Starbucks                     Starbucks   8655 Hiram Acworth Hwy   
2  Costco Pharmacy                        Costco  1715 Charles Hardy Pkwy   
3  Walgreens Photo  Walgreens;Walgreens Pharmacy     303 Charlie Watts Dr   
4       McDonald's                    McDonald's      3036 Villa Rica Hwy   

                  address_ds2 zip_ds1 zip_ds2  \
0   138 West Ellendale Avenue   97338   97338   
1  8655 Hiram Acworth Highway   30157   30157   
2  1715 Charles Hardy Parkway   30157   30157   
3     303 Charlie Watts Drive   30157   30157   
4     3036 Villa Rica Highway   30132   30132   

                                           mapbox_id  
0  dXJuOm1ieHBvaTpkOTZiZDM4MC1hMjg1LTQ2ZTEtYWVhYS...  
1  dXJuOm1ieHBvaTpjOTc4Y2NhMy0yODE5LTRlNWItODU4ZC...  
2  dXJuOm1ieHBvaTpjYzFmNjljNy0

In [9]:
merged_df.head(10)

,id_ds1,provider_ds1,name_ds1,address_ds1,geometry_ds1,categories_ds1,city_ds1,country_ds1,postcode_ds1,mapbox_id,...,address_ds2,geometry_ds2,categories_ds2,city_ds2,country_ds2,postcode_ds2,zip_ds2,name_embedding_ds2,address_embedding_ds2,categories_embedding_ds2
0,1,Provider 1,Starbucks,137 W Ellendale Ave,POINT (-123.31266147 44.93074775),cafe;coffee;coffee_shop;food;food_and_drink,NaN,US,973381408,dXJuOm1ieHBvaTpkOTZiZDM4MC1hMjg1LTQ2ZTEtYWVhYS...,...,138 West Ellendale Avenue,POINT (-123.31257629394531 44.930686950683594),bakery;cafe;coffee;coffee_shop;food;food_and_d...,Dallas,US,97338,97338,"[-0.010414955, -0.05963674, 0.0029248286, 0.13...","[0.042363793, 0.02030805, -0.046424385, -0.033...","[0.07029938, -0.06650684, -0.0015931061, 0.050..."
1,2,Provider 1,Starbucks,8655 Hiram Acworth Hwy,POINT (-84.75483938 34.00969773),grocery;shopping;supermarket,NaN,US,301576599,dXJuOm1ieHBvaTpjOTc4Y2NhMy0yODE5LTRlNWItODU4ZC...,...,8655 Hiram Acworth Highway,POINT (-84.75482940673828 34.00975036621094),bakery;cafe;coffee;coffee_shop;food;food_and_d...,Dallas,US,30157,30157,"[-0.010414955, -0.05963674, 0.0029248286, 0.13...","[-0.05119609, -0.0068536024, -0.043045696, 0.0...","[0.07029938, -0.06650684, -0.0015931061, 0.050..."
2,3,Provider 1,Costco Pharmacy,1715 Charles Hardy Pkwy,POINT (-84.76380003 33.92055333),health_services;pharmacy;services,NaN,US,30157,dXJuOm1ieHBvaTpjYzFmNjljNy0zMTQ1LTRlOTAtOTlhNy...,...,1715 Charles Hardy Parkway,POINT (-84.76396942138672 33.920631408691406),shopping;wholesale_store,Dallas,US,30157,30157,"[-0.052266948, 0.028888812, -0.055989027, 0.06...","[-0.00030382175, 0.06348002, -0.008515305, 0.0...","[-0.044802945, -0.025269827, -0.037189078, 0.0..."
3,4,Provider 1,Walgreens Photo,303 Charlie Watts Dr,POINT (-84.8237871 33.90817476),photo_store;photographer;services;shopping,NaN,US,30157,dXJuOm1ieHBvaTozZDU0YjBiNy1kYjlmLTQ4N2YtODc1Zi...,...,303 Charlie Watts Drive,POINT (-84.82364654541016 33.9081916809082),health_services;pharmacy;services,Dallas,US,30157,30157,"[-0.06581881, 0.039093018, -0.032595154, -0.03...","[-0.08947637, -0.035700493, -0.032218765, 0.04...","[-0.06817374, 0.036610164, -0.06953571, -0.112..."
4,5,Provider 1,McDonald's,3036 Villa Rica Hwy,POINT (-84.84245562 33.8677287),fast_food;food;food_and_drink,NaN,US,30132,dXJuOm1ieHBvaToxM2Q2Y2E4OC1jNDA2LTQ4NTItYWJiOS...,...,3036 Villa Rica Highway,POINT (-84.84228515625 33.86784362792969),breakfast_restaurant;burger_restaurant;fast_fo...,Dallas,US,30132,30132,"[-0.015024297, -0.06638837, 0.06632269, 0.0501...","[-0.030996421, -0.06624824, -0.103147395, 0.00...","[0.018335937, -0.028042816, 0.011874595, 0.030..."
5,6,Provider 1,McDonald's,1808 S Beltline Rd,POINT (-96.60095653 32.67676233),fast_food;food;food_and_drink,NaN,US,75253,dXJuOm1ieHBvaTo5YjIyMDk3Ny02OTQwLTRiYWItYTdlMC...,...,1808 South Beltline Road,POINT (-96.60086822509766 32.6767463684082),breakfast_restaurant;burger_restaurant;fast_fo...,Dallas,US,75253,75253,"[-0.015024297, -0.06638837, 0.06632269, 0.0501...","[-0.094799854, 0.011555125, -0.021132173, 0.03...","[0.018335937, -0.028042816, 0.011874595, 0.030..."
6,7,Provider 1,Starbucks,10305 Ferguson Road,POINT (-96.67351976 32.83520441),cafe;coffee;coffee_shop;food;food_and_drink,NaN,US,752283014,dXJuOm1ieHBvaTo3ZWE4MTJmNC0yM2E0LTQ1N2MtOWRlNy...,...,10305 Ferguson Road,POINT (-96.67353820800781 32.83521270751953),bakery;cafe;coffee;coffee_shop;food;food_and_d...,Dallas,US,75228,75228,"[-0.010414955, -0.05963674, 0.0029248286, 0.13...","[-0.05485619, -0.09025491, -0.025356982, 0.017...","[0.07029938, -0.06650684, -0.0015931061, 0.050..."
7,8,Provider 1,McDonald's,8055 S Loop 12,POINT (-96.68482078 32.71300254),fast_food;food;food_and_drink,NaN,US,75217,dXJuOm1ieHBvaTpjZGVhOGM2Zi1hNzZjLTQ5YjQtODJkMi...,...,8055 Great Trinity Forest Way,POINT (-96.68486022949219 32.71302795410156),health_services;pharmacy;services,Dallas,US,75217,75217,"[-0.015024297, -0.06638837, 0.06632269, 0.0501...","[0.07767395, 0.065604, -0.02467321, 0

### Exploring ways to get pairs so we can compare (join key not using mapbox_id)

Lets try a few appraoches:
* create a key based on name tokens + zip 
* Learn an embedding using training data (from the 100 samples) and uses ANN and the fact that datapoint needs to come from diff provider to get related businesses and match them 

In [ ]:
df1_with_embeddings["hash"] = 

count       100
unique       38
top       75206
freq          6
Name: zip, dtype: object